## Summary of the notebook

## Mount the drive 

In [1]:
# mount the drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Setting the correct path

In [2]:
import os

project_path = '/content/drive/MyDrive/MasterThesis'
task_name = 'CaliforniaHousingDatasetTests'
exp_name = 'GBP_explanation'
uncert_name = 'deep_ensemble'

%cd /content/drive/MyDrive/MasterThesis/ 

print(os.getcwd())
path = project_path + '/' + task_name + '/' + exp_name + '/' + uncert_name + '/' # 'CaliforniaHousingDatasetTests/GBP_explanation/flipout/'
print(path)

/content/drive/MyDrive/MasterThesis
/content/drive/MyDrive/MasterThesis
/content/drive/MyDrive/MasterThesis/CaliforniaHousingDatasetTests/GBP_explanation/deep_ensemble/


## Imports

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf

import os
import random 
import math

from keras.layers import *
from keras.models import Model, Sequential
import keras.backend as K

tf.compat.v1.disable_eager_execution()
print(tf.__version__)
print(f'Eager execution enabled : {tf.executing_eagerly()}')

import sys
sys.path.insert(0,'/content/drive/My Drive/MasterThesis/src/')

from utils import *

2.8.0
Eager execution enabled : False


## regression_nll_loss

In [4]:
import tensorflow as tf 

def regression_gaussian_nll_loss(variance_tensor, epsilon=1e-8, variance_logits=False):
    """
        Gaussian negative log-likelihood for regression, with variance estimated by the model.
        This function returns a keras regression loss, given a symbolic tensor for the sigma square output of the model.
        The training model should return the mean, while the testing/prediction model should return the mean and variance.
    """
    def nll(y_true, y_pred):
        #if variance_logits:
        #    variance_tensor = K.exp(variance_tensor)

        return 0.5 * tf.math.reduce_mean(tf.math.log(variance_tensor + epsilon) + tf.math.square(y_true - y_pred) / (variance_tensor + epsilon))

    return nll

## Load data

In [5]:
train_data, train_labels, val_data, val_labels, test_data, test_labels, feature_names = load_california_housing_data()
#print(train_labels[20])
#print(val_labels[30])
#print(test_labels[40])

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
Training data shape 
 (12750, 8)
Training labels shape 
 (12750, 1)
Validation data shape 
  (4250, 8)
Validation labels shape 
  (4250, 1)
Test data shape 
  (3000, 8)
Test labels shape 
  (3000, 1)


## Class definition

### DeepEnsemble

In [6]:
import keras
import numpy as np
import keras

import os
import yaml

from pydoc import locate
 
METADATA_FILENAME = "metadata.yml"

class DeepEnsemble:
    def __init__(self, model_fn=None, num_estimators=None, models=None, needs_test_estimators=False):
        self.needs_test_estimators = needs_test_estimators
        self.model_fn = model_fn
        self.num_estimators = num_estimators
        self.models = models

        if models is None:
            assert model_fn is not None and num_estimators is not None
            assert num_estimators > 0
            
            self.num_estimators = num_estimators
            self.train_estimators = [None] * num_estimators 
            self.test_estimators = [None] * num_estimators

            print('train_estimators ', self.train_estimators)
            print('test_estimators ', self.test_estimators)
            print('num_estimators ', self.num_estimators)
            

            for i in range(self.num_estimators):
                if self.needs_test_estimators:
                    estimators = model_fn()

                    if type(estimators) is not tuple:
                        raise ValueError("model_fn should return a tuple")

                    if len(estimators) != 2:
                        raise ValueError("model_fn returned a tuple of unexpected size ({} vs 2)".format(len(estimators)))

                    train_est, test_est = estimators
                    self.train_estimators[i] = train_est
                    self.test_estimators[i] = test_est
                else:
                    est = model_fn()
                    print('len of est ', len(est))
                    self.train_estimators[i] = est[0]
                    self.test_estimators[i] = est[1]
                    #print(f'train_estimators[{i}] : {est}')
                    #print(f'test_estimators[{i}] : {est}')

        else:
            if (model_fn is None and num_estimators is None):   # assert raises error with integer value for num_estimators   # https://stackoverflow.com/questions/46850472/assert-using-python-how-do-i-check-if-the-input-is-integer-or-not
                raise AssertionError('assign values to model_fn and num_estimators')

            self.train_estimators = models
            self.test_estimators = models
            self.num_estimators = len(models)


    def save(self, folder, filename_pattern="model-ensemble-{}.hdf5"):
        """
            Save a Deep Ensemble into a folder, using individual HDF5 files for each ensemble member.
            This allows for easily loading individual ensembles. Metadata is saved to allow loading of the whole ensemble.
        """

        if not os.path.exists(folder):
            os.makedirs(folder)

        model_metadata = {}

        for i in range(self.num_estimators):
            filename = os.path.join(folder, filename_pattern.format(i))
            self.test_estimators[i].save(filename)

            print("Saved estimator {} to {}".format(i, filename))

            model_metadata[i] = filename_pattern.format(i)

        metadata = {"models": model_metadata, "class": self.__module__}

        with open(os.path.join(folder, METADATA_FILENAME), 'w') as outfile:
            yaml.dump(metadata, outfile)
            

    @staticmethod
    def load(folder):
        """
            Load a Deep Ensemble model from a folder containing individual HDF5 files.
        """
        metadata = {}

        with open(os.path.join(folder, METADATA_FILENAME)) as infile:
            metadata = yaml.full_load(infile)

        models = []

        for _, filename in metadata["models"].items():
            models.append(keras.models.load_model(os.path.join(folder, filename)))

        clazz = locate(metadata["class"])

        return clazz(models=models)  

### DeepEnsembleRegressor

In [7]:
class DeepEnsembleRegressor(DeepEnsemble):
    """
        Implementation of a Deep Ensemble for regression.
        Uses two models, one for training and another for inference/testing. The user has to provide a model function that returns
        the train and test models, and use the provided deep_ensemble_nll_loss for training.
    """
    def __init__(self, model_fn=None, num_estimators=None, models=None):
        """
            Builds a Deep Ensemble given a function to make model instances, and the number of estimators.
            For training it uses a model that only outputs the mean, while the loss uses both the mean and variance produced by the model.
            For testing, a model that shares weights with the training model is used, but the testing model outputs both mean and variance. The final
            prediction is made with a mixture of gaussians, where each gaussian is one trained model instance.
        """
        super().__init__(model_fn=model_fn, num_estimators=num_estimators, models=models,
                         needs_test_estimators=True)
        
    def summary(self):
        print('training model summary ')
        print(self.model_fn()[0].summary())
        print('xxxxxxxxxxxxxxx')
        print('prediction model summary ')
        print(self.model_fn()[1].summary())
        

    def fit(self, X, y, epochs=10, batch_size=32, **kwargs):
        """
            Fits the Deep Ensemble, each estimator is fit independently on the same data.
        """

        for i in range(self.num_estimators):
            history = self.train_estimators[i].fit(X, y, epochs=epochs, batch_size=batch_size, **kwargs)

            # plotting the training and validation curves
            plt.plot(history.history['loss'], label='train loss')
            plt.plot(history.history['val_loss'], label='val loss')
            plt.legend()
            plt.grid()
            plt.xlabel('epochs')
            plt.ylabel('loss')
            plt.title('loss curves')
            plt.show()

            plt.plot(history.history['mae'], label='train mae')
            plt.plot(history.history['val_mae'], label='val mae')
            plt.legend()
            plt.grid()
            plt.xlabel('epochs')
            plt.ylabel('mae')
            plt.title('mae curves')
            plt.show()
            
    
    def fit_generator(self, generator, epochs=10, **kwargs):
        """
            Fits the Deep Ensemble, each estimator is fit independently on the same data.
        """

        for i in range(self.num_estimators):
            self.train_estimators[i].fit_generator(generator, epochs=epochs, **kwargs)
            

    def predict_output(self, X, batch_size=32, output_scaler=None, num_ensembles=None, disentangle_uncertainty=False, **kwargs):
        """
            Makes a prediction. Predictions from each estimator are used to build a gaussian mixture and its mean and standard deviation returned.
        """
        
        means = []
        variances = []

        if num_ensembles is None:
            estimators = self.test_estimators
        else:
            estimators = self.test_estimators[:num_ensembles]

        if "verbose" not in kwargs:
            kwargs["verbose"] = 0

        for estimator in estimators:
            mean, var  = estimator.predict(X, batch_size=batch_size, **kwargs)

            if output_scaler is not None:
                mean = output_scaler.inverse_transform(mean)

                # This should work but not sure if its 100% correct
                # Its not clear how to do inverse scaling of the variance
                sqrt_var = np.sqrt(var)
                var = output_scaler.inverse_transform(sqrt_var)
                var = np.square(var)

            means.append(mean)
            variances.append(var)

        means = np.array(means)
        variances = np.array(variances)
        
        mixture_mean = np.mean(means, axis=0)
        print('mixture_mean shape \n', mixture_mean.shape)
        print('mixture_mean \n', mixture_mean)
        mixture_var  = np.mean(variances + np.square(means), axis=0) - np.square(mixture_mean)
        mixture_var[mixture_var < 0.0] = 0.0
        print('mixture_var shape \n', mixture_var.shape)
        print('mixture_var \n', mixture_var)
                
        if disentangle_uncertainty:
            epi_var = np.var(means, axis=0)
            ale_var = np.mean(variances, axis=0)

            return mixture_mean, np.sqrt(ale_var), np.sqrt(epi_var)

        return mixture_mean, np.sqrt(mixture_var)

    def predict_generator(self, generator, steps=None, num_ensembles=None, **kwargs):
        """
            Makes a prediction. Predictions from each estimator are used to build a gaussian mixture and its mean and standard deviation returned.
        """
        
        means = []
        variances = []

        if num_ensembles is None:
            estimators = self.test_estimators
        else:
            estimators = self.test_estimators[:num_ensembles]

        for estimator in estimators:
            mean, var  = estimator.predict_generator(generator, steps=steps, **kwargs)
            means.append(mean)
            variances.append(var)

        means = np.array(means)
        variances = np.array(variances)
        
        mixture_mean = np.mean(means, axis=0)
        mixture_var  = np.mean(variances + np.square(means), axis=0) - np.square(mixture_mean)
        mixture_var[mixture_var < 0.0] = 0.0
                
        return mixture_mean, np.sqrt(mixture_var)

## Load the saved model

In [8]:
# slightly different procedure to load the ensemble model from yaml file 

import os

'''
dir_name = '/content/drive/MyDrive/MasterThesis/CaliforniaHousingDatasetTests/GBP_explanation/deep_ensemble/'
test = os.listdir(dir_name)

for item in test:
    if '.h5' in item:
        dir = dir_name+item
        with open(dir+'/model_as_json.json', 'r') as json_file:
            json_savedModel=json_file.read()
        model = tf.keras.models.model_from_json(json_savedModel)

model.summary()

# reference : https://towardsdatascience.com/saving-and-loading-keras-model-42195b92f57a
'''

model = [] 
num_estimators = 0 
for item in os.listdir(path):
    if '.h5' in item:
        print(path+item)
        for i in os.listdir(path+item):
            print(i)
            if '.hdf5' in i:
                model_variable = tf.keras.models.load_model(path+item+'/'+i)
                model.append(model_variable)
        #print('summary for estimator # ', num_estimators)
        #print(model[num_estimators].summary)
        num_estimators +=1

print('total number of ensemble components : ', len(model))

/content/drive/MyDrive/MasterThesis/CaliforniaHousingDatasetTests/GBP_explanation/deep_ensemble/ensemble_model_epochs_5_num_estimators_5.h5
model-ensemble-0.hdf5
model-ensemble-1.hdf5
model-ensemble-2.hdf5
model-ensemble-3.hdf5
model-ensemble-4.hdf5
metadata.yml
total number of ensemble components :  5


## Defining model_fn()

In [9]:
import keras 

from keras.layers import *
from keras.models import Model, Sequential
import keras.backend as K 

#obtained from hyperparameter optimization
def model_function():
    inp = Input(shape=(8,))
    x = Dense(32, activation='relu')(inp)
    x = Dense(32, activation='relu')(x)
    mean = Dense(1, activation='relu')(x)
    var = Dense(1, activation='softplus')(x)

    train_model = Model(inp, mean)
    pred_model = Model(inp, [mean, var])
    #train_model.compile(loss='mse', optimizer='sgd', metrics=['mae'])   
    train_model.compile(loss=regression_gaussian_nll_loss(var), optimizer='adam', metrics=['mae'])


    return train_model, pred_model

## Inference on test set

In [11]:

# Analysis of the input 
#num_of_inputs_to_be_explained = 1

#start_index = np.random.randint(0, test_data.shape[0])
#print('start_index : ', start_index)

#test_input = test_data[start_index:start_index+num_of_inputs_to_be_explained]
#print('test_input shape : ', test_input.shape)

#test_label = test_data[start_index:start_index+num_of_inputs_to_be_explained]
#print('test_label : ', test_data[start_index:start_index+num_of_inputs_to_be_explained])


#test_input_adj = np.expand_dims(test_input, axis=-1)
#print('test_input_adj shape :', test_input_adj.shape)


# MODEL PREDICTION AND PLOTTING 
stochastic_model = DeepEnsembleRegressor(model_fn=model_function, num_estimators=len(model), models=model)
pred_mean, pred_std = stochastic_model.predict_output(test_data, num_ensembles=len(model)) #we calculate mix_Var but return sqrt of it hence catch it in pred_std

print('pred_mean shape ', pred_mean.shape)
print('pred_std shape ', pred_std.shape)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


mixture_mean shape 
 (3000, 1)
mixture_mean 
 [[0.5053374 ]
 [0.27416188]
 [0.35749024]
 ...
 [0.1446454 ]
 [0.17725742]
 [0.7458336 ]]
mixture_var shape 
 (3000, 1)
mixture_var 
 [[0.15106606]
 [0.05928881]
 [0.08749378]
 ...
 [0.02757661]
 [0.035914  ]
 [0.35274774]]
pred_mean shape  (3000, 1)
pred_std shape  (3000, 1)
